In [ ]:
#import necessary libraries
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras import models 

In [ ]:
#Run this cell only if your data resides on Google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Set base_path to the location where the data and results of your project
#reside
base_path = '/content/gdrive/MyDrive/HPT/'

In [ ]:
#Decide the downstream task for which you want to perform
Dateset = "CE-MRI" 

In [ ]:
# Image shape set to constant for further use
if Dateset = "CE-MRI":
  IMG_SHAPE = (256, 256, 3)
elif Dateset = "HMS":
  IMG_SHAPE = (256, 256, 3)
elif Dataset = "Kaggle BTTC":
  

#Loading Saved Models for Different Strategies
The following functions will load the models as per the strategy provided in the arguments. 

Function to load ResNet50 base architecture as per the given input_shape

In [ ]:
def load_base_model(input_shape):
  base_model = tf.keras.applications.ResNet50(input_shape=input_shape, 
                                               include_top=False, 
                                               weights="imagenet")
  return base_model

Function to load models for the strategies based on the first one block of ResNet50 base architecture 

In [ ]:
#This function loads model based on first one block of ResNet50
#The parameter DAPT is of type boolean and indicates whether or 
#not DAPT was performed on the model.
#True value means that DAPT was performed (DA_TF_F1B strategy), False
#means that DAPT was not performed (ImageNet_TF_F1B strategy) 
def load_F1B_model(DAPT, input_shape):
  base_model = load_base_model(input_shape)
  
  img_input = layers.Input(shape=input_shape)

  x = img_input
  for i, layer in enumerate(base_model.layers):
    if i >= 1 and i<= 12:
      x = layer(x)
      if i == 6:
        temp1 = x
    elif i == 13:
      shortcut1 = layer(temp1)
    elif i == 14:
      x = layer(x)
    elif i == 15:
      shortcut1 = layer(shortcut1)
    elif i == 16:
      x = layer(x)
    elif i == 17:
      x = layers.add([x, shortcut1])
    elif i >= 18 and i <= 26:
      x = layer(x)
      if i == 18:
        temp2 = x
    elif i == 27:
      x = layers.add([x, temp2])
    elif i >= 28 and i <= 36:
      x = layer(x)
      if i == 28:
        temp3 = x
    elif i == 37:
      x = layers.add([x, temp3]) 
    elif i==38:
      x = layer(x)
  
  x = GlobalAveragePooling2D(name='avg_pool')(x)
  
  cloned_model = models.Model(img_input, x, name='TF_F1B')

  if DAPT:  #load the weights of DAPT trained model
    DA_TF_F1B_model = models.load_model(base_path + 'DAPT/Checkpoints/DA_TF_F1B/phase_2/checkpoint-150.h5')
    
    #remove the sigmoid layer
    DA_TF_F1B_model_base = tf.keras.models.Model(inputs=exp_4_model.input,
                              outputs=exp_4_model.layers[-2].output)

    #copy the weights of DAPT trained model to the newly created model
    cloned_model.set_weights(DA_TF_F1B_model_base.get_weights()) 
  else:  #copy ImageNet weights to the pruned model
    
    for i, layer in enumerate(base_model.layers):
    
      for j, trans_layer in enumerate(cloned_model.layers): 
        if layer.name==trans_layer.name:
          cloned_model.layers[j].set_weights(base_model.layers[i].get_weights())
      
  #return the model
  return cloned_model

Function to load models for the strategies based on the first two blocks of ResNet50 base architecture 

In [ ]:
#This function loads model based on first two blocks of ResNet50
#The parameter DAPT is of type boolean and indicates whether or 
#not DAPT was performed on the model.
#True value means that DAPT was performed (DA_TF_F2B strategy), False
#means that DAPT was not performed (ImageNet_TF_F2B strategy) 
def load_F2B_model(DAPT, input_shape):
  base_model = load_base_model(input_shape)

  img_input = layers.Input(shape=, input_shape)
  x = img_input

  for i, layer in enumerate(base_model.layers):
    if i >= 1 and i<= 12:
      x = layer(x)
      if i == 6:
        temp1 = x
    elif i == 13:
      shortcut1 = layer(temp1)
    elif i == 14:
      x = layer(x)
    elif i == 15:
      shortcut1 = layer(shortcut1)
    elif i == 16:
      x = layer(x)
    elif i == 17:
      x = layers.add([x, shortcut1])
    elif i >= 18 and i <= 26:
      x = layer(x)
      if i == 18:
        temp2 = x
    elif i == 27:
      x = layers.add([x, temp2])
    elif i >= 28 and i <= 36:
      x = layer(x)
      if i == 28:
        temp3 = x
    elif i == 37:
      x = layers.add([x, temp3]) 
    elif i >= 38 and i <= 44:
      x = layer(x)
      if i == 38:
        temp4 = x
    elif i == 45:
      shortcut2 = layer(temp4)
    elif i == 46: 
      x = layer(x)
    elif i == 47:
      shortcut2 = layer(shortcut2)
    elif i == 48:
      x = layer(x)
    elif i == 49:
      x = layers.add([x, shortcut2])
    elif i >= 50 and i <= 58:
      x = layer(x)
      if i == 50:
        temp5 = x
    elif i == 59:
      x = layers.add([x, temp5])
    elif i >= 60 and i <= 68:
      x = layer(x)
      if i == 60:
        temp6 = x
    elif i == 69:
      x = layers.add([x, temp6])
    elif i >= 70 and i <= 78:
      x = layer(x)
      if i == 70:
        temp7 = x
    elif i == 79:
      x = layers.add([x, temp7])
    elif i == 80:
      x = layer(x)
  
  x = GlobalAveragePooling2D(name='avg_pool')(x)
  cloned_model = models.Model(img_input, x, name='trans_two_block_resnet50')

  if DAPT:  #load the weights of DAPT trained model
    DA_TF_F2B_model = models.load_model(base_path + 'DAPT/Checkpoints/DA_TF_F2B/phase_2/checkpoint-150.h5')
    
    #remove the sigmoid layer
    DA_TF_F2B_model = tf.keras.models.Model(inputs=exp_4_model.input,
                              outputs=exp_4_model.layers[-2].output)

    #copy the weights of DAPT trained model to the newly created model
    cloned_model.set_weights(DA_TF_F2B_model.get_weights()) 
  else:  #copy ImageNet weights to the pruned model
    
    for i, layer in enumerate(base_model.layers):
    
      for j, trans_layer in enumerate(cloned_model.layers): 
        if layer.name==trans_layer.name:
          cloned_model.layers[j].set_weights(base_model.layers[i].get_weights())
  
  return cloned_model

Function to load models for the strategies based on the complete ResNet50 base architecture 

In [ ]:
#This function loads model for the strategies based on the complete ResNet50
# architecture. The parameter strategy is of type string and indicates the 
#strategy for which the corresponding checkpoint/saved model will be loaded.
 
def load_complete_architecture_model(strategy, input_shape):
  base_model = load_base_model(input_shape)
  global_average_layer = GlobalAveragePooling2D()(base_model.output)
  cloned_model = models.Model(inputs=base_model.input,
                              outputs=global_average_layer)
  
  #load model for partial DAPT strategies
  if strategy = "DA" or strategy = "DA_L1SB" or strategy = "DA_L2SB":
    strategy_model = models.load_model(base_path + 'DAPT/Checkpoints/' + Strategy + '/phase_2/checkpoint-150.h')
  
  #load model for hybrid DAPT strategies
  elif strategy = "DA_L1SB_PFT" or strategy = "DA_L2SB_PFT":
    strategy_model = models.load_model(base_path + 'DAPT/Checkpoints/' + Strategy + '/phase_3/checkpoint-50.h')
  
  #for ImageNet strategy, ImageNet weights will be used 
  if strategy = "ImageNet":
    
  else: #for other strategies, copy DAPT trained weights to the model
    cloned_model.set_weights(strategy_model.get_weights()) 
  
  return cloned_model

#Functions for Feature Extraction 
The following functions will extract the features from the loaded models for each dataset (where dataset name is provided as a function parameters)  

Extract features for brain tumor type classification with 3 classes (Meningioma, Glioma and Pituitary) for the datasets CE-MRI and Kaggle BTTC

In [ ]:
def extract_BTTC_features(dataset, strategy):
  
  #Extracting and Saving Glioma Features
  Glioma = np.load(base_path + 'Datasets/' + dataset '/Glioma.npy')
  print ("Glioma Examples Loaded")

  if strategy = "DA_TF_F1B":
    Glioma_predictions = DA_TF_F1B_model.predict(Glioma)
  elif strategy = "ImageNet_TF_F1B":
    Glioma_predictions = ImageNet_TF_F1B_model.predict(Glioma)
  elif strategy = "DA_TF_F2B":
    Glioma_predictions = DA_TF_F2B_model.predict(Glioma)
  elif strategy = "ImageNet_TF_F2B":
    Glioma_predictions = ImageNet_TF_F2B_model.predict(Glioma)
  elif strategy = "DA":
    Glioma_predictions = DA_model.predict(Glioma)
  elif strategy = "DA_L1SB":
    Glioma_predictions = DA_L1SB_model.predict(Glioma)
  elif strategy = "DA_L2SB":
    Glioma_predictions = DA_L2SB_model.predict(Glioma)
  elif strategy = "DA_L1SB_PFT":
    Glioma_predictions = DA_L1SB_PFT_model.predict(Glioma)
  elif strategy = "DA_L2SB_PFT":
    Glioma_predictions = DA_L2SB_PFT_model.predict(Glioma)
  elif strategy = "ImageNet":
    Glioma_predictions = ImageNet_model.predict(Glioma)

  #saving Glioma Features
  np.save(base_path + 'Features/' + strategy + '/' dataset + '/Glioma_Features.npy', Glioma_predictions)
  print ("Glioma Features Saved")

  ##################################################################
  #Extracting and Saving Meningioma Features
  Meningioma = np.load(base_path + 'Datasets/' + dataset '/Meningioma.npy')
  print ("Meningioma Examples Loaded")

  if strategy = "DA_TF_F1B":
    Meningioma_predictions = DA_TF_F1B_model.predict(Meningioma)
  elif strategy = "ImageNet_TF_F1B":
    Meningioma_predictions = ImageNet_TF_F1B_model.predict(Meningioma)
  elif strategy = "DA_TF_F2B":
    Meningioma_predictions = DA_TF_F2B_model.predict(Meningioma)
  elif strategy = "ImageNet_TF_F2B":
    Meningioma_predictions = ImageNet_TF_F2B_model.predict(Meningioma)
  elif strategy = "DA":
    Meningioma_predictions = DA_model.predict(Meningioma)
  elif strategy = "DA_L1SB":
    Meningioma_predictions = DA_L1SB_model.predict(Meningioma)
  elif strategy = "DA_L2SB":
    Meningioma_predictions = DA_L2SB_model.predict(Meningioma)
  elif strategy = "DA_L1SB_PFT":
    Meningioma_predictions = DA_L1SB_PFT_model.predict(Meningioma)
  elif strategy = "DA_L2SB_PFT":
    Meningioma_predictions = DA_L2SB_PFT_model.predict(Meningioma)
  elif strategy = "ImageNet":
    Meningioma_predictions = ImageNet_model.predict(Meningioma)

  #saving Meningioma Features
  np.save(base_path + 'Features/' + strategy + '/' dataset + '/Meningioma_Features.npy', Meningioma_predictions)
  print ("Meningioma Features Saved")

  ##################################################################
  #Extracting and Saving Pituitary Features
  Pituitary = np.load(base_path + 'Datasets/' + dataset '/Pituitary.npy')
  print ("Pituitary Examples Loaded")

  if strategy = "DA_TF_F1B":
    Pituitary_predictions = DA_TF_F1B_model.predict(Pituitary)
  elif strategy = "ImageNet_TF_F1B":
    Pituitary_predictions = ImageNet_TF_F1B_model.predict(Pituitary)
  elif strategy = "DA_TF_F2B":
    Pituitary_predictions = DA_TF_F2B_model.predict(Pituitary)
  elif strategy = "ImageNet_TF_F2B":
    Pituitary_predictions = ImageNet_TF_F2B_model.predict(Pituitary)
  elif strategy = "DA":
    Pituitary_predictions = DA_model.predict(Pituitary)
  elif strategy = "DA_L1SB":
    Pituitary_predictions = DA_L1SB_model.predict(Pituitary)
  elif strategy = "DA_L2SB":
    Pituitary_predictions = DA_L2SB_model.predict(Pituitary)
  elif strategy = "DA_L1SB_PFT":
    Pituitary_predictions = DA_L1SB_PFT_model.predict(Pituitary)
  elif strategy = "DA_L2SB_PFT":
    Pituitary_predictions = DA_L2SB_PFT_model.predict(Pituitary)
  elif strategy = "ImageNet":
    Pituitary_predictions = ImageNet_model.predict(Pituitary)

  #saving Pituitary Features
  np.save(base_path + 'Features/' + strategy + '/' dataset + '/Pituitary_Features.npy', Pituitary_predictions)
  print ("Pituitary Features Saved")

Extract features for brain disease classification with 5 classes (Inflammatory, Cerebrovascular, Degenerative, Normal and Neoplastic) for the dataset HMS

In [ ]:
def extract_brain_disease_features(strategy):
  #Extracting and Saving Inflammatory Features
  Inflammatory = np.load(base_path + 'Datasets/HMS/Inflammatory.npy')
  print ("Inflammatory Examples Loaded")

  if strategy = "DA_TF_F1B":
    Inflammatory_predictions = DA_TF_F1B_model.predict(Inflammatory)
  elif strategy = "ImageNet_TF_F1B":
    Inflammatory_predictions = ImageNet_TF_F1B_model.predict(Inflammatory)
  elif strategy = "DA_TF_F2B":
    Inflammatory_predictions = DA_TF_F2B_model.predict(Inflammatory)
  elif strategy = "ImageNet_TF_F2B":
    Inflammatory_predictions = ImageNet_TF_F2B_model.predict(Inflammatory)
  elif strategy = "DA":
    Inflammatory_predictions = DA_model.predict(Inflammatory)
  elif strategy = "DA_L1SB":
    Inflammatory_predictions = DA_L1SB_model.predict(Inflammatory)
  elif strategy = "DA_L2SB":
    Inflammatory_predictions = DA_L2SB_model.predict(Inflammatory)
  elif strategy = "DA_L1SB_PFT":
    Inflammatory_predictions = DA_L1SB_PFT_model.predict(Inflammatory)
  elif strategy = "DA_L2SB_PFT":
    Inflammatory_predictions = DA_L2SB_PFT_model.predict(Inflammatory)
  elif strategy = "ImageNet":
    Inflammatory_predictions = ImageNet_model.predict(Inflammatory)

  #saving Inflammatory Features
  np.save(base_path + 'Features/' + strategy + '/HMS/Inflammatory_Features.npy', Inflammatory_predictions)
  print ("Inflammatory Features Saved")

  #############################################################################
  #Extracting and Saving Cerebrovascular Features
  Cerebrovascular = np.load(base_path + 'Datasets/HMS/Cerebrovascular.npy')
  print ("Cerebrovascular Examples Loaded")

  if strategy = "DA_TF_F1B":
    Cerebrovascular_predictions = DA_TF_F1B_model.predict(Cerebrovascular)
  elif strategy = "ImageNet_TF_F1B":
    Cerebrovascular_predictions = ImageNet_TF_F1B_model.predict(Cerebrovascular)
  elif strategy = "DA_TF_F2B":
    Cerebrovascular_predictions = DA_TF_F2B_model.predict(Cerebrovascular)
  elif strategy = "ImageNet_TF_F2B":
    Cerebrovascular_predictions = ImageNet_TF_F2B_model.predict(Cerebrovascular)
  elif strategy = "DA":
    Cerebrovascular_predictions = DA_model.predict(Cerebrovascular)
  elif strategy = "DA_L1SB":
    Cerebrovascular_predictions = DA_L1SB_model.predict(Cerebrovascular)
  elif strategy = "DA_L2SB":
    Cerebrovascular_predictions = DA_L2SB_model.predict(Cerebrovascular)
  elif strategy = "DA_L1SB_PFT":
    Cerebrovascular_predictions = DA_L1SB_PFT_model.predict(Cerebrovascular)
  elif strategy = "DA_L2SB_PFT":
    Cerebrovascular_predictions = DA_L2SB_PFT_model.predict(Cerebrovascular)
  elif strategy = "ImageNet":
    Cerebrovascular_predictions = ImageNet_model.predict(Cerebrovascular)

  #saving Cerebrovascular Features
  np.save(base_path + 'Features/' + strategy + '/HMS/Cerebrovascular_Features.npy', Cerebrovascular_predictions)
  print ("Cerebrovascular Features Saved")

  #############################################################################
  #Extracting and Saving Degenerative Features
  Degenerative = np.load(base_path + 'Datasets/HMS/Degenerative.npy')
  print ("Degenerative Examples Loaded")

  if strategy = "DA_TF_F1B":
    Degenerative_predictions = DA_TF_F1B_model.predict(Degenerative)
  elif strategy = "ImageNet_TF_F1B":
    Degenerative_predictions = ImageNet_TF_F1B_model.predict(Degenerative)
  elif strategy = "DA_TF_F2B":
    Degenerative_predictions = DA_TF_F2B_model.predict(Degenerative)
  elif strategy = "ImageNet_TF_F2B":
    Degenerative_predictions = ImageNet_TF_F2B_model.predict(Degenerative)
  elif strategy = "DA":
    Degenerative_predictions = DA_model.predict(Degenerative)
  elif strategy = "DA_L1SB":
    Degenerative_predictions = DA_L1SB_model.predict(Degenerative)
  elif strategy = "DA_L2SB":
    Degenerative_predictions = DA_L2SB_model.predict(Degenerative)
  elif strategy = "DA_L1SB_PFT":
    Degenerative_predictions = DA_L1SB_PFT_model.predict(Degenerative)
  elif strategy = "DA_L2SB_PFT":
    Degenerative_predictions = DA_L2SB_PFT_model.predict(Degenerative)
  elif strategy = "ImageNet":
    Degenerative_predictions = ImageNet_model.predict(Degenerative)

  #saving Degenerative Features
  np.save(base_path + 'Features/' + strategy + '/HMS/Degenerative_Features.npy', Degenerative_predictions)
  print ("Degenerative Features Saved")

  #############################################################################
  #Extracting and Saving Normal Features
  Normal = np.load(base_path + 'Datasets/HMS/Normal.npy')
  print ("Normal Examples Loaded")

  if strategy = "DA_TF_F1B":
    Normal_predictions = DA_TF_F1B_model.predict(Normal)
  elif strategy = "ImageNet_TF_F1B":
    Normal_predictions = ImageNet_TF_F1B_model.predict(Normal)
  elif strategy = "DA_TF_F2B":
    Normal_predictions = DA_TF_F2B_model.predict(Normal)
  elif strategy = "ImageNet_TF_F2B":
    Normal_predictions = ImageNet_TF_F2B_model.predict(Normal)
  elif strategy = "DA":
    Normal_predictions = DA_model.predict(Normal)
  elif strategy = "DA_L1SB":
    Normal_predictions = DA_L1SB_model.predict(Normal)
  elif strategy = "DA_L2SB":
    Normal_predictions = DA_L2SB_model.predict(Normal)
  elif strategy = "DA_L1SB_PFT":
    Normal_predictions = DA_L1SB_PFT_model.predict(Normal)
  elif strategy = "DA_L2SB_PFT":
    Normal_predictions = DA_L2SB_PFT_model.predict(Normal)
  elif strategy = "ImageNet":
    Normal_predictions = ImageNet_model.predict(Normal)

  #saving Normal Features
  np.save(base_path + 'Features/' + strategy + '/HMS/Normal_Features.npy', Normal_predictions)
  print ("Normal Features Saved")

  #############################################################################
  #Extracting and Saving Neoplastic Features
  Neoplastic = np.load(base_path + 'Datasets/HMS/Neoplastic.npy')
  print ("Neoplastic Examples Loaded")

  if strategy = "DA_TF_F1B":
    Neoplastic_predictions = DA_TF_F1B_model.predict(Neoplastic)
  elif strategy = "ImageNet_TF_F1B":
    Neoplastic_predictions = ImageNet_TF_F1B_model.predict(Neoplastic)
  elif strategy = "DA_TF_F2B":
    Neoplastic_predictions = DA_TF_F2B_model.predict(Neoplastic)
  elif strategy = "ImageNet_TF_F2B":
    Neoplastic_predictions = ImageNet_TF_F2B_model.predict(Neoplastic)
  elif strategy = "DA":
    Neoplastic_predictions = DA_model.predict(Neoplastic)
  elif strategy = "DA_L1SB":
    Neoplastic_predictions = DA_L1SB_model.predict(Neoplastic)
  elif strategy = "DA_L2SB":
    Neoplastic_predictions = DA_L2SB_model.predict(Neoplastic)
  elif strategy = "DA_L1SB_PFT":
    Neoplastic_predictions = DA_L1SB_PFT_model.predict(Neoplastic)
  elif strategy = "DA_L2SB_PFT":
    Neoplastic_predictions = DA_L2SB_PFT_model.predict(Neoplastic)
  elif strategy = "ImageNet":
    Neoplastic_predictions = ImageNet_model.predict(Neoplastic)

  #saving Neoplastic Features
  np.save(base_path + 'Features/' + strategy + '/HMS/Neoplastic_Features.npy', Neoplastic_predictions)
  print ("Neoplastic Features Saved")

#Function Calls to Extract and Save Features for All the Strategies

Function Calls to Extract and Save Features for the Strategy DA_TF_F1B

In [ ]:
########################################################
#load/set the model for DA_TF_F1B for HMS/CE-MRI
Image_shape = (256, 256, 3)
DA_TF_F1B_model = load_F1B_model(True, Image_shape)

#extract and save CE-MRI features 
extract_BTTC_features("CE-MRI", "DA_TF_F1B")

#extract and save HMS features
extract_brain_disease_features("HMS", "DA_TF_F1B")

########################################################
#load/set the model for DA_TF_F1B for Kaggle BTTC dataset
Image_shape = (240, 240, 3)
DA_TF_F1B_model = load_F1B_model(True, Image_shape)

#extract and save Kaggle BTTC features
extract_BTTC_features("Kaggle BTTC", "DA_TF_F1B")

#ImageNet_TF_F1B_model = load_F1B_model(False)

#load/set the model for TF_F2B
DA_TF_F2B_model = load_F2B_model(True)
ImageNet_TF_F2B_model = load_F2B_model(False)

#load the model for DA strategy
DA_model = load_complete_architecture_model("DA")

#load the model for DA_L1SB strategy
DA_L1SB_model = load_complete_architecture_model("DA_L1SB")

#load the model for DA_L2SB strategy
DA_L2SB_model = load_complete_architecture_model("DA_L2SB")

#load the model for DA_L1SB_PFT strategy
DA_L1SB_PFT_model = load_complete_architecture_model("DA_L1SB_PFT")

#load the model for DA_L2SB_PFT strategy
DA_L2SB_PFT_model = load_complete_architecture_model("DA_L2SB_PFT")

#load the model for ImageNet strategy
ImageNet_model = load_complete_architecture_model("ImageNet")

Function Calls to Extract and Save Features for the Strategy ImageNet_TF_F1B



In [ ]:
########################################################
#load/set the model for ImageNet_TF_F1B for HMS/CE-MRI
Image_shape = (256, 256, 3)
ImageNet_TF_F1B_model = load_F1B_model(False, Image_shape)

#extract and save CE-MRI features 
extract_BTTC_features("CE-MRI", "ImageNet_TF_F1B")

#extract and save HMS features
extract_brain_disease_features("HMS", "ImageNet_TF_F1B")

########################################################
#load/set the model for DA_TF_F1B for Kaggle BTTC dataset
Image_shape = (240, 240, 3)
ImageNet_TF_F1B_model = load_F1B_model(False, Image_shape)

#extract and save Kaggle BTTC features
extract_BTTC_features("Kaggle BTTC", "ImageNet_TF_F1B")

Function Calls to Extract and Save Features for the Strategy DA_TF_F2B

In [ ]:
########################################################
#load/set the model for DA_TF_F2B for HMS/CE-MRI
Image_shape = (256, 256, 3)
DA_TF_F2B_model = load_F2B_model(True, Image_shape)

#extract and save CE-MRI features 
extract_BTTC_features("CE-MRI", "DA_TF_F2B")

#extract and save HMS features
extract_brain_disease_features("HMS", "DA_TF_F2B")

########################################################
#load/set the model for DA_TF_F2B for Kaggle BTTC dataset
Image_shape = (240, 240, 3)
DA_TF_F2B_model = load_F2B_model(True, Image_shape)

#extract and save Kaggle BTTC features
extract_BTTC_features("Kaggle BTTC", "DA_TF_F2B")

Function Calls to Extract and Save Features for the Strategy ImageNet_TF_F2B

In [ ]:
########################################################
#load/set the model for ImageNet_TF_F2B for HMS/CE-MRI
Image_shape = (256, 256, 3)
ImageNet_TF_F2B_model = load_F2B_model(False, Image_shape)

#extract and save CE-MRI features 
extract_BTTC_features("CE-MRI", "ImageNet_TF_F2B")

#extract and save HMS features
extract_brain_disease_features("HMS", "ImageNet_TF_F2B")

########################################################
#load/set the model for ImageNet_TF_F2B for Kaggle BTTC dataset
Image_shape = (240, 240, 3)
ImageNet_TF_F2B_model = load_F2B_model(False, Image_shape)

#extract and save Kaggle BTTC features
extract_BTTC_features("Kaggle BTTC", "ImageNet_TF_F2B")

Function Calls to Extract and Save Features for the Strategy DA

In [ ]:
###############################################
#load the model for DA strategy for HMS/CE-MRI
Image_shape = (256, 256, 3)
DA_model = load_complete_architecture_model("DA", Image_shape)

#extract and save CE-MRI features 
extract_BTTC_features("CE-MRI", "DA")

#extract and save HMS features
extract_brain_disease_features("HMS", "DA")

########################################################
#load/set the model for DA for Kaggle BTTC dataset
Image_shape = (240, 240, 3)
DA_model = load_complete_architecture_model("DA", Image_shape)

#extract and save Kaggle BTTC features
extract_BTTC_features("Kaggle BTTC", "DA")

Function Calls to Extract and Save Features for the Strategy DA_L1SB




In [ ]:
###############################################
#load the model for DA_L1SB strategy for HMS/CE-MRI
Image_shape = (256, 256, 3)
DA_model = load_complete_architecture_model("DA_L1SB", Image_shape)

#extract and save CE-MRI features 
extract_BTTC_features("CE-MRI", "DA_L1SB")

#extract and save HMS features
extract_brain_disease_features("HMS", "DA_L1SB")

########################################################
#load/set the model for DA_L1SB for Kaggle BTTC dataset
Image_shape = (240, 240, 3)
DA_model = load_complete_architecture_model("DA_L1SB", Image_shape)

#extract and save Kaggle BTTC features
extract_BTTC_features("Kaggle BTTC", "DA_L1SB")

Function Calls to Extract and Save Features for the Strategy DA_L2SB




In [ ]:
###############################################
#load the model for DA_L2SB strategy for HMS/CE-MRI
Image_shape = (256, 256, 3)
DA_model = load_complete_architecture_model("DA_L2SB", Image_shape)

#extract and save CE-MRI features 
extract_BTTC_features("CE-MRI", "DA_L2SB")

#extract and save HMS features
extract_brain_disease_features("HMS", "DA_L2SB")

########################################################
#load/set the model for DA_L2SB for Kaggle BTTC dataset
Image_shape = (240, 240, 3)
DA_model = load_complete_architecture_model("DA_L2SB", Image_shape)

#extract and save Kaggle BTTC features
extract_BTTC_features("Kaggle BTTC", "DA_L2SB")

Function Calls to Extract and Save Features for the Strategy DA_L1SB_PFT

In [ ]:
###############################################
#load the model for DA_L1SB_PFT strategy for HMS/CE-MRI
Image_shape = (256, 256, 3)
DA_model = load_complete_architecture_model("DA_L1SB_PFT", Image_shape)

#extract and save CE-MRI features 
extract_BTTC_features("CE-MRI", "DA_L1SB_PFT")

#extract and save HMS features
extract_brain_disease_features("HMS", "DA_L1SB_PFT")

########################################################
#load/set the model for DA_L1SB_PFT for Kaggle BTTC dataset
Image_shape = (240, 240, 3)
DA_model = load_complete_architecture_model("DA_L1SB_PFT", Image_shape)

#extract and save Kaggle BTTC features
extract_BTTC_features("Kaggle BTTC", "DA_L1SB_PFT")

Function Calls to Extract and Save Features for the Strategy DA_L2SB_PFT

In [ ]:
###############################################
#load the model for DA_L2SB_PFT strategy for HMS/CE-MRI
Image_shape = (256, 256, 3)
DA_model = load_complete_architecture_model("DA_L2SB_PFT", Image_shape)

#extract and save CE-MRI features 
extract_BTTC_features("CE-MRI", "DA_L2SB_PFT")

#extract and save HMS features
extract_brain_disease_features("HMS", "DA_L2SB_PFT")

########################################################
#load/set the model for DA_L2SB_PFT for Kaggle BTTC dataset
Image_shape = (240, 240, 3)
DA_model = load_complete_architecture_model("DA_L2SB_PFT", Image_shape)

#extract and save Kaggle BTTC features
extract_BTTC_features("Kaggle BTTC", "DA_L2SB_PFT")

Function Calls to Extract and Save Features for the Strategy ImageNet

In [ ]:
###############################################
#load the model for ImageNet strategy for HMS/CE-MRI
Image_shape = (256, 256, 3)
DA_model = load_complete_architecture_model("ImageNet", Image_shape)

#extract and save CE-MRI features 
extract_BTTC_features("CE-MRI", "ImageNet")

#extract and save HMS features
extract_brain_disease_features("HMS", "ImageNet")

########################################################
#load/set the model for ImageNet for Kaggle BTTC dataset
Image_shape = (240, 240, 3)
DA_model = load_complete_architecture_model("ImageNet", Image_shape)

#extract and save Kaggle BTTC features
extract_BTTC_features("Kaggle BTTC", "ImageNet")

In [ ]:
#Extract and Save Features (CE-MRI)
extract_BTTC_features("CE-MRI", "DA_TF_F1B")
extract_BTTC_features("CE-MRI", "ImageNet_TF_F1B")
extract_BTTC_features("CE-MRI", "DA_TF_F2B")
extract_BTTC_features("CE-MRI", "ImageNet_TF_F2B")
extract_BTTC_features("CE-MRI", "DA")
extract_BTTC_features("CE-MRI", "DA_L1SB")
extract_BTTC_features("CE-MRI", "DA_L2SB")
extract_BTTC_features("CE-MRI", "DA_L1SB_PFT")
extract_BTTC_features("CE-MRI", "DA_L2SB_PFT")
extract_BTTC_features("CE-MRI", "ImageNet")

In [ ]:
#Extract and Save Features (Kaggle BTTC)
extract_BTTC_features("Kaggle BTTC", "DA_TF_F1B")
extract_BTTC_features("Kaggle BTTC", "ImageNet_TF_F1B")
extract_BTTC_features("Kaggle BTTC", "DA_TF_F2B")
extract_BTTC_features("Kaggle BTTC", "ImageNet_TF_F2B")
extract_BTTC_features("Kaggle BTTC", "DA")
extract_BTTC_features("Kaggle BTTC", "DA_L1SB")
extract_BTTC_features("Kaggle BTTC", "DA_L2SB")
extract_BTTC_features("Kaggle BTTC", "DA_L1SB_PFT")
extract_BTTC_features("Kaggle BTTC", "DA_L2SB_PFT")
extract_BTTC_features("Kaggle BTTC", "ImageNet")

In [ ]:
#Extract and Save Features (HMS)
extract_brain_disease_features("DA_TF_F1B")
extract_brain_disease_features("ImageNet_TF_F1B")
extract_brain_disease_features("DA_TF_F2B")
extract_brain_disease_features("ImageNet_TF_F2B")
extract_brain_disease_features("DA")
extract_brain_disease_features("DA_L1SB")
extract_brain_disease_features("DA_L2SB")
extract_brain_disease_features("DA_L1SB_PFT")
extract_brain_disease_features("DA_L2SB_PFT")
extract_brain_disease_features("ImageNet")